## Incorporate Tiffany's comments in [PR 1370](https://github.com/cal-itp/data-analyses/pull/1370)

In [1]:
import _section1_utils as section1
import geopandas as gpd
import merge_data
import merge_operator_data
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers
from shared_utils import portfolio_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [2]:
import sys

sys.path.append("../gtfs_funnel/")
import route_typologies

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
organization_name = "Capitol Corridor Joint Powers Authority"

In [5]:
org_name_lists = ["Capitol Corridor Joint Powers Authority"]

In [6]:
analysis_date_list = ["2024-11-13"]

In [7]:
analysis_date = "2024-11-13"

In [8]:
name_list = ["Bay Area 511 Capitol Corridor Schedule"]

In [9]:
schd_keys = [
    "f5a749dd65924e025b1293c58f95f8d6",
]

In [13]:
roads = route_typologies.prep_roads(GTFS_DATA_DICT)

In [14]:
TYPOLOGY_THRESHOLD = 0.09

In [15]:
ROAD_BUFFER_METERS = 20

In [16]:
gdf = route_typologies.overlay_shapes_to_roads(roads, analysis_date, ROAD_BUFFER_METERS)

In [17]:
gdf.columns

Index(['schedule_gtfs_dataset_key', 'route_id', 'direction_id',
       'common_shape_id', 'route_name', 'route_meters', 'freq_category',
       'typology', 'pct_typology'],
      dtype='object')

In [18]:
gdf.head(1)

,schedule_gtfs_dataset_key,route_id,direction_id,common_shape_id,route_name,route_meters,freq_category,typology,pct_typology
0,0139b1253130b33adcd4b3a4490530d2,0177a66b-9f33-407d-a72e-776429fb73d4,0.00,7fc3b128-6535-4bde-8635-e79706254f45,70,54486.44,high,rapid,0.02


In [25]:
gdf_train = gdf.loc[gdf.route_name.str.contains("train")]

In [26]:
gdf_train[['route_name']].drop_duplicates()

,route_name
5503,California Ave Caltrain
6929,San Mateo Caltrain - College of San Mateo
6951,San Carlos Caltrain - Carlmont Village
6963,Redwood City Caltrain - Florence/17th loop
6982,Redwood City Caltrain - Canada College
7230,Burlingame School - Carolan (Burlingame Caltrain)
7368,San Carlos Caltrain - Alameda/Ralston
7445,Redwood City Caltrain - Woodside HS
7509,Sharon Heights - Menlo Park Caltrain
8425,Gilroy Caltrain and Gavilan College


In [20]:
gdf.typology.unique()

array(['rapid', 'coverage', 'downtown_local', 'local'], dtype=object)

In [22]:
gdf2 = gdf.loc[gdf.schedule_gtfs_dataset_key.isin(schd_keys)]

In [27]:
SCHED_GCS

'gs://calitp-analytics-data/data-analyses/gtfs_schedule/'

In [29]:
GTFS_DATA_DICT.schedule_tables.route_typologies

'nacto_typologies/route_typologies'

In [36]:
dec_test = pd.read_parquet("gs://calitp-analytics-data/data-analyses/gtfs_schedule/nacto_typologies/route_typologies_2024-12-11.parquet")

In [37]:
dec_test.columns

Index(['schedule_gtfs_dataset_key', 'name', 'route_type', 'route_id',
       'route_long_name', 'route_short_name', 'combined_name', 'is_express',
       'is_rapid', 'is_rail', 'is_local', 'direction_id', 'common_shape_id',
       'route_name', 'route_meters', 'is_coverage', 'is_downtown_local'],
      dtype='object')

In [38]:
dec_test.loc[dec_test.schedule_gtfs_dataset_key.isin(schd_keys)]

,schedule_gtfs_dataset_key,name,route_type,route_id,route_long_name,route_short_name,combined_name,is_express,is_rapid,is_rail,is_local,direction_id,common_shape_id,route_name,route_meters,is_coverage,is_downtown_local
1545,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,3,SF,Shuttle_SF,SF,SF__Shuttle_SF,0,1,0,0,0.00,cpz2,Shuttle to San Francisco Transbay Terminal,13495.52,0,1
1546,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,3,Shuttle,Shuttle_Auburn,Shuttle,Shuttle__Shuttle_Auburn,0,0,0,0,1.00,104dc91b-4f7b-4f93-bea2-ebfe9adabcf3,Shuttle to Auburn,60220.51,1,0
1547,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,2,CC,Capitol Corridor,CC,CC__Capitol Corridor,0,1,1,1,0.00,5cf6811a-2f53-4199-b315-4408eb816e82,"Daily train service between Auburn, Sacramento, Oakland and San Jose",143715.40,0,0
1548,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,2,CC,Capitol Corridor,CC,CC__Capitol Corridor,0,0,1,0,1.00,92232e9e-55e8-4f8a-bb86-1aeb515be83e,"Daily train service between Auburn, Sacramento, Oakland and San Jose",143684.90,1,0
